In [1]:
import matplotlib.pyplot as plt
import torch
import logging


from power_perceiver.data_loader.data_loader import BatchKey
from power_perceiver.data_loader.satellite_from_zarr import SatelliteDataset, worker_init_fn


logging.basicConfig()
_log = logging.getLogger("power_perceiver")
_log.setLevel(logging.DEBUG)

plt.rcParams["figure.figsize"] = (20, 12)

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SATELLITE_ZARR_PATH = (
    "gs://public-datasets-eumetsat-solar-forecasting/satellite/EUMETSAT/SEVIRI_RSS/v3/eumetsat_seviri_hrv_uk.zarr")

In [5]:
torch.manual_seed(42)    

sat_dataset = SatelliteDataset(satellite_zarr_path=SATELLITE_ZARR_PATH)

train_dataloader = torch.utils.data.DataLoader(
    sat_dataset,
    batch_size=32,
    num_workers=2,  # TODO: Change?
    pin_memory=False,  # TODO: Change.
    worker_init_fn=worker_init_fn,
)

In [6]:
%%time
for batch in train_dataloader:
    break

INFO:power_perceiver:worker_id=0 has random number generator seed=6,909,045,637,428,952,499
INFO:power_perceiver:worker_id=1 has random number generator seed=6,909,045,637,428,952,500
INFO:power_perceiver:Before any selection, there are 621 days of data from 2020-01-01 00:05:00 to 2021-11-07 15:50:00
INFO:power_perceiver:Before any selection, there are 621 days of data from 2020-01-01 00:05:00 to 2021-11-07 15:50:00
INFO:power_perceiver:After filtering, there are 332 days of data from 2020-01-01 09:00:00 to 2020-12-31 15:55:00
INFO:power_perceiver:After filtering, there are 332 days of data from 2020-01-01 09:00:00 to 2020-12-31 15:55:00


CPU times: user 16.6 ms, sys: 28.1 ms, total: 44.7 ms
Wall time: 31.3 s


In [8]:
batch[BatchKey.hrvsatellite].shape

torch.Size([32, 31, 1, 64, 64])